In [ ]:
%reset -f
import glob, os, sys, inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import pandas as pd
from wavhandler import *
from natsort import natsorted
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import io
import PIL
np.random.seed(0)
plt.rcParams['figure.figsize'] = [24, 8]

import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)

In [ ]:
smpl = 1
z = WavHandler('/home/yannis/data/insects/Potamitis/Wingbeats/Aedes', sample_size=smpl, recursive=True)
z.read()
z.df_signals.iloc[:,0].plot()

In [ ]:
z.preprocess()
#z.filter_accepted_signals()
#print(len(z.accepted_signals))

In [ ]:
sig = z.df_signals.iloc[:,0]
from scipy.signal import find_peaks
f, Pxx = signal.welch(sig, 8000., scaling='density', window='hamming')
x = Pxx
threshold = 0.000000025
peaks, vals = find_peaks(x, height=threshold, distance=10)
peaks = [v for i,v in enumerate(peaks) if f[peaks][i] > 300]
plt.plot(f,Pxx);plt.plot(f[peaks], x[peaks], "x");plt.plot(np.zeros_like(x), "--", color="gray")
plt.xlabel('frequency [Hz]');plt.ylabel('PSD, hamming');plt.title('Power spectrum (scipy.signal.welch)')

df = pd.DataFrame(np.vstack((f[peaks], x[peaks])).T, columns=['freqs','pows'])
peakseries = df[df.freqs>400]['pows'].nlargest(5)
print('peakseries: \n',peakseries)
if peakseries.shape[0] == 2:
    # If there are 2 peaks, return True only if
    # the highest peak is larger than the threshold OR
    # the second highest peak is larger than the threshold
    condition = (peakseries.iloc[0] > threshold or peakseries.iloc[1] > threshold)
elif peakseries.shape[0] > 2:
    # If there are more peaks, return True if
    # the same as above AND
    # the leftmost peak is higher than the third peak (counting from the left) AND
    # the second peak from the left is also higher than the third peak
    condition = (peakseries.iloc[0] > threshold or peakseries.iloc[1] > threshold) and \
            (peakseries.loc[0] > peakseries.loc[2]) and \
            (peakseries.loc[1] > peakseries.loc[2])
else:
    condition = False
print(f[peaks])
print(condition)

In [ ]:

sigseries = pd.Series(sig)
window = 500
rolling_mean = np.abs(sigseries).rolling(window).mean()
sigseries.iloc[rolling_mean.idxmax()+1-window:rolling_mean.idxmax()+1].plot()

In [ ]:
freqs, times, spectrogram = signal.spectrogram(sig, 8000., noverlap=50,window='hamming')
plt.title('Spectrogram')
plt.ylabel('Frequency band')
plt.xlabel('Time window')
#plt.imshow(spectrogram, aspect='auto', cmap='hot_r', origin='lower')

plt.pcolormesh(times, freqs, spectrogram)

In [ ]:
plt.plot(spectrogram.sum(axis=0))#, keepdims=True))


In [ ]:
times, times.shape


In [ ]:
freqs,freqs.shape

In [ ]:
spectrogram

In [ ]:
spectrogram.shape